In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
import string
import re

In [6]:
data = pd.read_csv('US_Crime_Data.csv')
data.sample(10)

,Date,Title,Organization,City,State,URL,Keyword,Summary
6536,12/7/17 8:41,Jewish Federation's Executive Director Tapped ...,wmassjewishledger.com,Amherst,MA,http://www.wmassjewishledger.com/2017/12/jewis...,NaN,NaN
1042,4/16/17 13:16,"As Anti-LGBTQ Persecution Worsens in Russia, T...",Huffington Post,New York,NY,http://www.huffingtonpost.com/entry/as-anti-lg...,asylum baume blog chechnya electrocutions gay ...,The US government needs to publicly condemn th...
2682,6/19/17 7:53,"How To Be Proudly, Publicly & Safely Muslim Af...",atlantamuslim.com,Atlanta,GA,http://www.atlantamuslim.com/2017/jun/19/how-t...,NaN,NaN
1746,5/23/17 8:56,Describing victim as 'not a thug' is a problem...,Baltimore Sun,Baltimore,MD,http://www.baltimoresun.com/news/opinion/reade...,baltimore bowie collins crime hate iii lam mar...,NaN
2616,6/16/17 18:33,Police investigate anti-gay Bible verse carved...,KING5.com,Seattle,WA,http://www.king5.com/news/local/police-investi...,amendment bible carved facebook fence first ga...,NaN
4844,9/21/17 8:58,Police investigate graffiti found on Oakland t...,Milpitas Post,Milpitas,CA,http://www.mercurynews.com/2017/09/21/anti-sem...,albert area bay goers graffiti group hate jewi...,"Rabbi Jacqueline Mates-Muchin, left, and Rabbi..."
2671,6/19/17 5:56,Man Charged in Death of Virginia Muslim Teenager,Delmarva Public Radio,Salisbury,MD,http://delmarvapublicradio.net/post/man-charge...,area brandished charges darwin dulles hassanen...,NaN
3112,7/3/17 11:44,Hate crimes spike again in California; African...,Sacramento Bee,Sacramento,CA,http://www.sacbee.com/news/politics-government...,africa anti becerra california crime crimes do...,Law enforcement agencies reported Monday an 11...
3965,8/17/17 22:03,How to respond?,Fort Wayne Journal Gazette,Fort Wayne,IN,http://www.journalgazette.net/opinion/editoria...,amendment bias bosma brian charlottesville cri...,Indiana is one of five states without any form...
1779,5/24/17 7:51,Racist graffiti and vandalism leads to federal...,WCPO,Cincinnati,OH,http://www.wcpo.com/news/crime/racist-graffiti...,attorney crime east evicted glassman hate hill...,NaN


In [7]:
data.isna().sum()

Date               0
Title              1
Organization       0
City            1167
State           1245
URL                0
Keyword         1176
Summary         2256
dtype: int64

For this model we just need headlines

In [8]:
df = data[['Title']]
df = df.dropna().reset_index(drop = True)
df.head()

,Title
0,Pizza Hut driver who killed co-worker with sho...
1,Residents of NJ township receiving KKK promoti...
2,House OKs bill to expand Kentucky's hate crime...
3,"Amid Protests, 'Blue Lives Matter' Bill Passes..."
4,Lafourche inmates charged with hate crimes in ...


In [9]:
df['Title'][123]

'St. Louis suburb victimized by cemetery vandalism mulling hate crimes registry'

In [10]:
df.shape

(6782, 1)

Removing Punctuations

In [11]:
def clean_text(df):
    df['Title'] = df['Title'].apply(lambda x : x.lower())
    tokens = df['Title'].str.replace('[{}]'.format(string.punctuation), '')
    return tokens

In [12]:
tokens = clean_text(df)

C:\Users\Nuwanga Wijamuni\AppData\Local\Temp\ipykernel_10740\363633391.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  tokens = df['Title'].str.replace('[{}]'.format(string.punctuation), '')


In [13]:
tokens

0       pizza hut driver who killed coworker with shot...
1       residents of nj township receiving kkk promoti...
2       house oks bill to expand kentuckys hate crimes...
3       amid protests blue lives matter bill passes ke...
4       lafourche inmates charged with hate crimes in ...
                              ...                        
6777    police rash of vandalism on margate menorahs i...
6778    exstudent accused of smearing used tampon on r...
6779    queensbury woman charged with hate crime again...
6780    hate crime hartford student accused of smearin...
6781    police make arrest following alleged hate crim...
Name: Title, Length: 6782, dtype: object

In [14]:
len(set(tokens))

6563

Tokenizing

In [15]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(tokens)
seq = tokenizer.texts_to_sequences(tokens)

In [16]:
seq[:10]

[[3973, 3974, 450, 47, 88, 3975, 12, 2211, 103, 1370, 2803],
 [830, 4, 423, 1558, 2212, 470, 3976, 672],
 [257, 1846, 75, 5, 593, 3977, 1, 6, 36],
 [721, 898, 480, 274, 301, 75, 481, 634, 257],
 [3978, 2804, 16, 12, 1, 6, 3, 424, 19],
 [13, 722, 1, 2, 48, 594, 3979, 5, 401],
 [3980, 480, 274, 301, 75, 21, 1371, 2805, 5, 214, 3981, 173],
 [194, 320, 18, 402, 2806, 239, 15, 98, 38],
 [194,
  320,
  18,
  402,
  2806,
  239,
  15,
  98,
  38,
  635,
  174,
  556,
  557,
  14,
  498,
  558,
  275,
  42],
 [33, 174, 673, 302, 2, 1372, 2807, 3, 2213]]

In [17]:
x = []
y = []
total_words_drop = 0
for i in seq:
    if len(i) > 1:
        for j in range(1, len(i)):
            x.append(i[:j])
            y.append(i[j])
            
    else : 
        total_words_drop +=1
print('Total Words Dropped : {}'.format(total_words_drop))

Total Words Dropped : 12


In [18]:
y[: 10]

[3974, 450, 47, 88, 3975, 12, 2211, 103, 1370, 2803]

Padding sequences

In [19]:
x = tf.keras.preprocessing.sequence.pad_sequences(x)

In [20]:
x.shape


(64701, 49)

Shaping y same as x

In [21]:
y = tf.keras.utils.to_categorical(y)

In [22]:
y.shape

(64701, 7569)

Vocab Size : total no. of unique words

In [23]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

7569

LSTM Model

In [24]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size,49 ),
                            tf.keras.layers.LSTM(100, return_sequences = True),
                            tf.keras.layers.LSTM(100),
                            tf.keras.layers.Dense(100, activation = 'relu'),
                            tf.keras.layers.Dense(vocab_size, activation = 'softmax')])

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 49)          370881    
                                                                 
 lstm (LSTM)                 (None, None, 100)         60000     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 7569)              764469    
                                                                 
Total params: 1,285,850
Trainable params: 1,285,850
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.compile(loss  = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'],
             )

In [27]:
history = model.fit(x,y,
                   epochs = 100,
                    batch_size = 256,
                    callbacks = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                                               patience = 5,
                                                               restore_best_weights = True))

Epoch 1/100
253/253 [==============================] - 92s 346ms/step - loss: 6.9955 - accuracy: 0.0611
Epoch 2/100
217/253 [========================>.....] - ETA: 13s - loss: 6.4754 - accuracy: 0.0955

KeyboardInterrupt: 

In [28]:
model.save('model.h5')

In [29]:
vocab_array = np.array(list(tokenizer.word_index.keys()))
vocab_array

array(['hate', 'crime', 'in', ..., 'walks', 'lgbti', 'retires'],
      dtype='<U23')

Final Function for Predictions

In [30]:
def make_predictions(text, n_words):
    for i in range(n_words):
        text_tokenize = tokenizer.texts_to_sequences([text])
        text_padding  = tf.keras.preprocessing.sequence.pad_sequences(text_tokenize, maxlen = 49)
        prediction = np.squeeze(np.argmax(model.predict(text_padding), axis = -1))
        prediction = str(vocab_array[prediction - 1])
        text += " " + prediction
    return text

Testing Model

In [31]:
make_predictions('california',5)

1/1 [==============================] - 0s 28ms/step


'california hate crime hate crime hate'

In [32]:
make_predictions('new york',8)

1/1 [==============================] - 0s 32ms/step


'new york hate crime hate crime hate crime hate crime'

In [33]:
make_predictions('highway',8)

1/1 [==============================] - 0s 45ms/step


'highway hate crime hate crime hate crime hate crime'

In [ ]:
make_predictions('home',8)

1/1 [==============================] - 0s 20ms/step


'home charged with hate crime in attack on elderly'

In [ ]:
make_predictions('pizza',12)

1/1 [==============================] - 0s 23ms/step


'pizza hut driver who killed coworker with shotgun found mentally competent to rant'

In [ ]:
make_predictions('allegations',9)

1/1 [==============================] - 0s 26ms/step


'allegations hate crimes are the start rate law but true'